***
***

#### Тестовое задание в ChemTech AI

##### Исполнитель: Друзык Роман Богданович

##### Задание
 * Используя данные представленные по ссылке Train_Data_200k создать модель, нацеленную на прогнозирование значений параметров Target_1...4 по значениям Tag_1...79.
 * После получения модели применить ее к тестовой выборке представленной в архиве ML_Data в файле test_data_100k и на основании значений Tag_1...79 получить прогнозы для параметров Target_1...4.
 * Полученные прогнозы и топ-10 значимых тэгов направить в качестве результата в ответном письме.

***

#### *Проведем анализ данных*

*Загрузим необходимы для построения модели библиотеки*

In [375]:
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
import plotly.graph_objects as go
import plotly.express as px
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostRegressor, Pool
%matplotlib inline
warnings.filterwarnings('ignore')

*Загрузим обучающий тестовый датасеты*

In [309]:
train = pd.read_csv(r'C:\datasets\ChemTechAI\train_data_200k.csv')
test = pd.read_csv(r'C:\datasets\ChemTechAI\test_data_100k.csv')

*Посмотрим визуально на данные*

In [310]:
train.head(3)

,Unnamed: 0,tag1,tag2,tag3,tag4,tag5,tag6,tag7,tag8,tag9,...,tag74,tag75,tag76,tag77,tag78,tag79,target1,target2,target3,target4
0,2016-08-18 00:00:00,15.78042,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,53.02723,66.30531,55.10297,73.65612,NaN,NaN,NaN,NaN,NaN,NaN
1,2016-08-18 00:01:00,15.80157,NaN,NaN,NaN,NaN,NaN,53.0644,NaN,NaN,...,53.05109,66.30098,55.11534,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-08-18 00:02:00,15.68913,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,53.06495,66.30359,55.09395,NaN,NaN,55.07365,NaN,NaN,NaN,NaN


In [311]:
test.head(3)

,Unnamed: 0,tag1,tag2,tag3,tag4,tag5,tag6,tag7,tag8,tag9,...,tag70,tag71,tag72,tag73,tag74,tag75,tag76,tag77,tag78,tag79
0,2017-01-03 21:21:00,12.59972,NaN,NaN,NaN,NaN,NaN,45.27993,43.13644,NaN,...,37.63644,40.64410,53.44470,NaN,36.59567,64.47523,45.08920,NaN,NaN,NaN
1,2017-01-03 21:22:00,12.65850,NaN,NaN,NaN,NaN,NaN,NaN,42.98455,NaN,...,37.69201,40.68030,53.43953,NaN,36.60715,64.48697,45.18127,NaN,NaN,NaN
2,2017-01-03 21:23:00,12.69538,NaN,NaN,NaN,NaN,NaN,NaN,41.87401,NaN,...,37.71147,40.69286,53.42733,NaN,36.60194,64.48034,45.00529,75.84169,NaN,NaN


***

*Посмотрим на количество пропусков в обоих датасетах, относительно общего размера датасета*

In [312]:
train_NaN_view = train.isna().sum().sort_values(ascending=False) / train.shape[0]
test_NaN_view = test.isna().sum().sort_values(ascending=False) /  test.shape[0]

fig = go.Figure()
fig.add_trace(go.Bar(
    x=train_NaN_view.index,
    y=train_NaN_view.values,
    name='Train DataSet',
    marker_color='indianred'
))
fig.add_trace(go.Bar(
    x=test_NaN_view.index,
    y=test_NaN_view.values,
    name='Test DataSet',
    marker_color='lightsalmon'
))
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.show()

*Видим, что у довольно большого количества признаков очень много пропущеных значений*

*Порасуждаем на этот счет:*

*Мы работаем с данными, полученными с определенного набора датчиков, как каком-то из производств. Отсутствие значения в признаке(который является показаниями датчика) может означать, что датчик в данный момент не фиксировал каих-либо действий, так как они не происходили. В то же время, то небольшое количество полученных данных с датчиков, по которым пропусков 99%, может свидетельствовать о наступлении какого-то важного события в производственной цепочке.*

*Если построение дальнейшей модели мы будем основывать на данном предположении, то выкидывать показания с NaN из таблицы будет категорически не верно.*

*Так же заменять NaN любой эвристикой (нулями, средними, медианой и тд) можно посчитать плохой стратегией, так как отсутвие значения на датчике означает именно отсутвие значения, а не синтетическую эвристику*

*Поэтому в дальнейшем нашем анализе и выборе модели будем придерживаться именно такого предположения*

***

#### *Подготовим наши датасеты для модели машинного обучения*

*Установим признак даты как индекс*

In [313]:
train.index = pd.DatetimeIndex(train['Unnamed: 0'])
test.index = pd.DatetimeIndex(test['Unnamed: 0'])

*Удалим признак даты, так как он  стал индексом*

In [314]:
train = train.drop(['Unnamed: 0'], axis=1)
test = test.drop(['Unnamed: 0'], axis=1)

*Так же удалим из обучающего датасета события в которых по целевым признакам имеются NaN*

In [315]:
train = train.dropna(how='all', subset=['target1', 'target2', 'target3', 'target4'])

*Разобъем обучающий датасет на X и y*

In [316]:
X_train = train.drop(['target1', 'target2', 'target3', 'target4'], axis=1)
y_train = train[['target1', 'target2', 'target3', 'target4']]

***

#### Модель

*Для прогнозирования мы будем использовать CatBoostRegressor, как зарекомендовавший себя с хорошей стороны(мной были попробованы другие классификаторы, они оказались хуже чем CatBoost)*

*Для кросс-валидации применим sklearn.model_selection.TimeSeriesSplit, так как у нас имеется временная последовательность событий*

In [317]:
tscv = TimeSeriesSplit(n_splits=10)

*Подберем некторые гиперпараметры*

In [323]:
param_grid = {'learning_rate': [0.03, 0.06, 0.1], 'depth': [4, 6, 8], 'l2_leaf_reg': [3, 5, 7]}
model = CatBoostRegressor(loss_function='MultiRMSE', nan_mode='Min', silent=True)
grid_search = GridSearchCV(model, param_grid=param_grid, cv=tscv)

*Обучим модель*

In [324]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=10),
             error_score=nan,
             estimator=<catboost.core.CatBoostRegressor object at 0x000000004BA7AAC8>,
             iid='deprecated', n_jobs=None,
             param_grid={'depth': [4, 6, 8], 'l2_leaf_reg': [3, 5, 7],
                         'learning_rate': [0.03, 0.06, 0.1]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

*Обучение заняло сутки(нет GPU)*

*Посмотрим параметры лучшей модели по результатам решетчатого поиска*

In [329]:
grid_best_param = grid_search.best_params_
print("Параметры лучшей модели: {}".format(grid_best_param))

Параметры лучшей модели: {'depth': 4, 'l2_leaf_reg': 3, 'learning_rate': 0.03}


*Посмотрим на лучший результат, по результатам кросс-валидации*

In [340]:
grid_best_score = grid_search.best_estimator_.best_score_
print("Скор на обучающем наборе: {}".format(grid_best_score))

Скор на обучающем наборе: {'learn': {'MultiRMSE': 0.001283504560946091}}


*Получим предсказания на тестовом наборе*

In [350]:
predicted = pd.DataFrame(grid_search.best_estimator_.predict(test), columns=y_train.columns, index=test.index)

*Сохраним результат в файл*

In [352]:
predicted.to_csv(r'C:\datasets\ChemTechAI\test_predict.csv')

***

#### Посмотрим на важность признаков

*Сохраним важность признаков с именами в DataFrame, для удобства анализа*

In [360]:
feature_imortance_name = pd.DataFrame(data=[grid_search.best_estimator_.feature_names_, 
                                            grid_search.best_estimator_.feature_importances_]).T
feature_imortance_name.columns = ['tags', 'importance']

*Сохраним топ-10 признаков в отдельный файл*

In [374]:
feature_imortance_ratio = feature_imortance_name.sort_values(by='importance', ascending=False)
feature_imortance_ratio.head(10).to_csv(r'C:\datasets\ChemTechAI\top10_feature_importance.csv')

*Построим график на котором отобразим важность признаков в виде столбчатой диаграммы*

In [376]:
fig = px.bar(feature_imortance_ratio, y='importance', x='tags', text='tags')
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.show()